<a href="https://colab.research.google.com/github/DhivyaMadhavan/FaceRecognition_CV2_Mediapipe/blob/main/Proctoring.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install mediapipe opencv-python



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.7/35.7 MB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 12.9 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.15.0 requires protobuf<4.21,>=3.20.3; python_version < "3.11", but you have protobuf 4.25.4 which is incompatible.


#capture screenshots initially



The code captures 3 initial face images from the video

In [2]:
import cv2
import mediapipe as mp

def get_face_features(frame, face_landmarks):
    ih, iw, _ = frame.shape
    landmarks = np.array([
        [landmark.x * iw, landmark.y * ih] for landmark in face_landmarks.landmark
    ])
    return landmarks.flatten()  # Flatten to a 1D vector

def capture_initial_faces(video_path, num_samples=3):
    screenshots = []
    cap = cv2.VideoCapture(video_path)

    mp_face_mesh = mp.solutions.face_mesh.FaceMesh(
        max_num_faces=10,
        refine_landmarks=True,
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5
    )

    sample_count = 0

    while cap.isOpened() and sample_count < num_samples:
        ret, frame = cap.read()
        if not ret:
            break

        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = mp_face_mesh.process(rgb_frame)

        if results.multi_face_landmarks:
            for face_landmarks in results.multi_face_landmarks:
                screenshots.append(frame)
                sample_count += 1
                if sample_count >= num_samples:
                    break

    cap.release()

    # Save initial screenshots
    for i, screenshot in enumerate(screenshots):
        cv2.imwrite(f"initial_face_{i+1}.png", screenshot)

# Example usage
video_path = '/content/Untitled design (1).mp4'  # Replace with your video file path
capture_initial_faces(video_path)


/usr/local/lib/python3.10/dist-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


#with saved images , frame_skip = 3

In [3]:
import cv2
import mediapipe as mp
import numpy as np

def get_face_features(frame, face_landmarks):
    ih, iw, _ = frame.shape
    landmarks = np.array([
        [landmark.x * iw, landmark.y * ih] for landmark in face_landmarks.landmark
    ])
    return landmarks.flatten()  # Flatten to a 1D vector

def load_initial_face_features(image_paths):
    face_features = []
    mp_face_mesh = mp.solutions.face_mesh.FaceMesh(
        max_num_faces=1,
        refine_landmarks=True,
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5
    )

    for image_path in image_paths:
        image = cv2.imread(image_path)
        rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = mp_face_mesh.process(rgb_image)

        if results.multi_face_landmarks:
            for face_landmarks in results.multi_face_landmarks:
                features = get_face_features(image, face_landmarks)
                face_features.append(features)

    return face_features

def compare_face_features(initial_features, current_features, threshold=5500):
    for feature in initial_features:
        distance = np.linalg.norm(feature - current_features)
        if distance < threshold:
            return True
    return False

def process_frame(frame, face_mesh, initial_features, head_movement_threshold, last_face_positions, suspicious_activities):
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(rgb_frame)

    current_face_boxes = []
    detected_features = []

    if results.multi_face_landmarks:
        num_faces = len(results.multi_face_landmarks)
        if num_faces == 0:
            suspicious_activities['No Face Detected'] += 1
        elif num_faces > 1:
            suspicious_activities['Multiple Faces Detected'] += 1

        for face_landmarks in results.multi_face_landmarks:
            ih, iw, _ = frame.shape
            x_min = min(int(landmark.x * iw) for landmark in face_landmarks.landmark)
            x_max = max(int(landmark.x * iw) for landmark in face_landmarks.landmark)
            y_min = min(int(landmark.y * ih) for landmark in face_landmarks.landmark)
            y_max = max(int(landmark.y * ih) for landmark in face_landmarks.landmark)
            face_box = (x_min, y_min, x_max - x_min, y_max - y_min)
            current_face_boxes.append(face_box)

            # Extract features for face comparison
            features = get_face_features(frame, face_landmarks)
            detected_features.append(features)

    movement_detected = False
    if last_face_positions:
        for current_box in current_face_boxes:
            x, y, w, h = current_box
            for last_box in last_face_positions:
                lx, ly, lw, lh = last_box
                cx, cy = x + w // 2, y + h // 2
                lc_x, lc_y = lx + lw // 2, ly + lh // 2
                if np.sqrt((cx - lc_x) ** 2 + (cy - lc_y) ** 2) > head_movement_threshold:
                    movement_detected = True
                    break
            if movement_detected:
                break

    if movement_detected:
        suspicious_activities['Rapid Head Movement'] += 1

    # Check if the detected face features match the initial ones
    same_person_detected = False
    for feature in detected_features:
        if compare_face_features(initial_features, feature):
            same_person_detected = True
            break

    if not same_person_detected:
        suspicious_activities['isSamePerson'] = False

    return current_face_boxes

def process_video_for_faces_and_movement(video_path, initial_face_paths, head_movement_threshold=10, frame_skip=3):
    initial_face_features = load_initial_face_features(initial_face_paths)
    if not initial_face_features:
        print("No faces detected in the initial screenshots.")
        return

    suspicious_activities = {
        'Multiple Faces Detected': 0,
        'Rapid Head Movement': 0,
        'isSamePerson': True
    }

    cap = cv2.VideoCapture(video_path)
    mp_face_mesh = mp.solutions.face_mesh.FaceMesh(
        max_num_faces=10,
        refine_landmarks=True,
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5
    )

    last_face_positions = []
    frame_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_count += 1
        if frame_count % frame_skip != 0:
            continue

        last_face_positions = process_frame(frame, mp_face_mesh, initial_face_features, head_movement_threshold, last_face_positions, suspicious_activities)

    cap.release()

    # Print the summary of activities
    print("Activities summary:")
    for activity, count in suspicious_activities.items():
        if activity == 'isSamePerson':
            print(f"{activity}: {'True' if count else 'False'}")
        else:
            print(f"{activity}: {count} times")

# Example usage
video_path = '/content/Untitled design (1).mp4'  # Replace with your video file path
initial_face_paths = [
    '/content/initial_face_1.png',  # Replace with your initial face screenshot paths
    '/content/initial_face_2.png',
    '/content/initial_face_3.png'
]
process_video_for_faces_and_movement(video_path, initial_face_paths)


Activities summary:
Multiple Faces Detected: 0 times
Rapid Head Movement: 15 times
isSamePerson: True
